In [22]:
!git clone https://github.com/dmklee/image2sphere.git
!git clone https://github.com/DavidRuhe/clifford-group-equivariant-neural-networks.git

Cloning into 'image2sphere'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 169 (delta 17), reused 16 (delta 16), pack-reused 144 (from 1)
Receiving objects: 100% (169/169), 28.19 MiB | 39.82 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Cloning into 'clifford-group-equivariant-neural-networks'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 112 (delta 32), reused 23 (delta 23), pack-reused 67 (from 1)
Receiving objects: 100% (112/112), 349.95 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [32]:
import sys
import os
sys.path.append("/kaggle/working/image2sphere")
# original_path = "/kaggle/working/clifford-group-equivariant-neural-networks"
# new_path = "/kaggle/working/clifford"
# os.rename(original_path, new_path)
sys.path.append("/kaggle/working/clifford")

In [25]:
# !pip install -r /kaggle/working/image2sphere/requirements.txt

In [ ]:
# !wget ftp://cs.stanford.edu/cs/cvgl/PASCAL3D+_release1.1.zip

# Imports

## CGENN imports

In [35]:
from clifford.models.modules.linear import MVLinear
from clifford.models.modules.gp import SteerableGeometricProductLayer
from clifford.models.modules.mvlayernorm import MVLayerNorm
from clifford.models.modules.mvsilu import MVSiLU

## I2S imports

In [34]:
import image2sphere.src.pascal_dataset
from image2sphere.src.models import ResNet
from image2sphere.src.pascal_dataset import Pascal3D
from image2sphere.src.so3_utils import so3_healpix_grid

##  Other imports

In [38]:
import torch 
import torch.nn as nn 

# CGENN

## CGENN basic blocks

In [39]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()

        self.layers = nn.Sequential(
            MVLinear(algebra, in_features, out_features),
            MVSiLU(algebra, out_features),
            SteerableGeometricProductLayer(algebra, out_features),
            MVLayerNorm(algebra, out_features)
        )

    def forward(self, input):
        # [batch_size, in_features, 2**d] -> [batch_size, out_features, 2**d]
        return self.layers(input)

class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers=2):
        super().__init__()

        layers = []
        for i in range(n_layers - 1):
            layers.append(
                CGEBlock(algebra, in_features, hidden_features)
            )
            in_features = hidden_features

        layers.append(
            CGEBlock(algebra, hidden_features, out_features)
        )
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

## CGENN model

In [40]:
class EquvariantCGENN(nn.Module):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.cgemlp = CGEMLP(ca, in_features, hidden_features, hidden_features)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features)
        )

    def forward(self, input):
        h = self.cgemlp(input)
        # Index the hidden states at 0 to get the invariants, and let a regular MLP do the final processing.
        return self.mlp(h[..., 0])

# Final models

In [ ]:
class IGA2S(nn.Module):
    def __init__(
        self, 
        num_classes, # provided as part of the input
        train_grid_rec_level,
        eval_grid_rec_level,
        size=50, 
        pretrained=True,
        global_pooling=True,
        hidden_dim
    ):
        
        self.encoder = ResNet(size, pretrained, global_pooling)

        train_grid = so3.healpix_grid(rec_level=train_grid_rec_level)
        self.register_buffer('train_rotmats', o3.angles_to_matrix(*train_grid))
        

        eval_grid = so3.healpix_grid(rec_level=eval_grid_rec_level)
        self.register_buffer('eval_rotmats', o3.angles_to_matrix(*eval_grid))

        input_dim = self.encoder.output_shape[0]
        self.equiv = EquvariantCGENN(input_dim + num_classes, hidden_dim, )
        
        
        

## Model 1: ResNet + CGENN head

# Training

## Data prep

## Training & Evaluation functions

## Training